In [3]:
from __future__ import unicode_literals
import string
import re
import math
import os, os.path
import nltk
from nltk.stem import PorterStemmer 
import numpy as np
import copy
from nltk.tokenize import word_tokenize 

In [4]:
#bring stopwords from file 
docs = []
PositionIndexArray = []
InvertedIndexArray = []
docnum = 0
with open("stopwords.txt" , 'r', encoding="utf-8") as f:
    stops = []
    for line in f:
        line = line[:-1]
        stops.append(line)
f.close()

In [5]:
#get path of file for reading 
paths = []
title = "dataset/cars"
for (dirpath, dirnames, filenames) in os.walk(str(os.getcwd())+'/'+title+'/2007'):
    for i in filenames:
        paths.append(str(dirpath)+str("/")+i)
for (dirpath, dirnames, filenames) in os.walk(str(os.getcwd())+'/'+title+'/2008'):
    for i in filenames:
        paths.append(str(dirpath)+str("/")+i)
for (dirpath, dirnames, filenames) in os.walk(str(os.getcwd())+'/'+title+'/2009'):
    for i in filenames:
        paths.append(str(dirpath)+str("/")+i)    

In [6]:
#add every txet file as a doc to docs
def addDoc(text , name,docno):
    ch = True
    for doc in docs:
        if doc.id == name:
            ch = False
            break
    if ch:
        try:    
            docs.append(prepareData(text,name,docno))
        except:
            a = 5
    else:
        print('this doc has been added!!')

In [7]:
# pre process such as tokenize , convert numbers , convert to lower ,stemming , remove stopwords , remove single character and bad characters
def prepareData(text,name , docno):
    res = []
    lines = text
    lines = convert_numbers(lines)
    lines = convert_lower_case(lines)
    stemmer = PorterStemmer()
    lines = str(lines)
    lines = nltk.word_tokenize(lines)   
    line2 = []
    for w in lines:
        if w not in stops:
            w = stemmer.stem(w)
            line2.append(w)
    res.append(line2)
    res = remove_single_characters(res)
    res = remove_punctuation(res)
    doc = Doc(text,name,res,docno)
    return doc

In [8]:
#function for convert number
def convert_numbers(data):
    data = np.char.replace(data, "0", " zero ")
    data = np.char.replace(data, "1", " one ")
    data = np.char.replace(data, "2", " two ")
    data = np.char.replace(data, "3", " three ")
    data = np.char.replace(data, "4", " four ")
    data = np.char.replace(data, "5", " five ")
    data = np.char.replace(data, "6", " six ")
    data = np.char.replace(data, "7", " seven ")
    data = np.char.replace(data, "8", " eight ")
    data = np.char.replace(data, "9", " nine ")
    return data

In [9]:
#function for convert data to lower case
def convert_lower_case(data):
    return np.char.lower(data)

In [10]:
#function for remove single characters
def remove_single_characters(data):
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if len(w) > 1:
            new_text = new_text + " " + w
    return np.char.strip(new_text)

In [11]:
#function for remove ' from data
def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

In [12]:
#function for remover bad char
def remove_punctuation(data):
    symbols = "![\"#$%&()*+-./:;<=>?@\^_`{|}~] ' ''"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
    data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, "   ", " ")
    data = np.char.replace(data,"    ", " ")
    data = np.char.replace(data, ',', '')
    data = np.char.replace(data, '...', '')
    return data

In [13]:
#create a class for each text file
class Doc:
    def __init__(self,lines,id,main,num):
        self.lines = lines
        self.id = id
        self.main = main
        self.num = num

In [14]:
#use the path for reading files , preprocess and create class for each file 
#user path[57:] for name because path contain all directory and we nedd just name of file
#docnum is number of doc we seted for it
def readlAlldata():
    docno = 0
    for path in paths:
        file = open(path, 'r', encoding='cp1250')
        text = file.read().strip()
        name = path[56:]
        docno += 1
        file.close()
        addDoc(text, name , docno)

In [15]:
readlAlldata()

In [16]:
#get all words from docs that we create stop this specific word and remove duplicate words
allwords = []
common_words = ["docno","doc","date","author", "text", "favorit","zero","one","two" ,"three","four","five","six", "seven","eight","nine"] 
for doc in docs:
    temp = doc.main
    temp = str(temp)
    temp = remove_punctuation(temp)
    temp = str(temp)
    temp = remove_single_characters(temp)
    temp = str(temp)
    temp = temp.split()
    for word in temp:
        if word not in common_words:
            if word not in allwords:
                allwords.append(word)

In [17]:
#create a inverted index class for each word
class InInd:
    def __init__(self,word,abundance,posting):
        self.word = word
        self.abundance = abundance
        self.posting = posting

In [18]:
#create posting list for inverted index 
def addWordToWords1(word1):
    check = True
    for wo in InvertedIndexArray:
        if wo.word == word1:
            check = False
            print("word is checked!")
            break
    if check:
        posting=[]
        index = []
        docfreq = 0
        for doc in docs:
            checkInDoc = False
            temp = doc.main
            temp = str(temp)
            if word1 in temp:
                checkInDoc = True
                docfreq +=1
                if checkInDoc :
                    id = doc.num
                    posting.append(id)
        index = InInd(word1,docfreq,posting)
        InvertedIndexArray.append(index)

In [ ]:
#add all word to posting list
for i in allwords:
    i = remove_punctuation(i)
    i = str(i)
    addWordToWords1(i)

In [21]:
#write invertedt index in text file
with open("Inverted_index.txt" , 'w') as f:
    for i in InvertedIndexArray:
        f.write(str(i.word))
        f.write(" : ")
        f.write(str(i.abundance))
        f.write(" ------> ")
        f.write(str(i.posting))
        f.write("\n")

In [ ]:
#create post of all words DF and TF 
def addWordToWords(word1):
    check = True
    for wo in PositionIndexArray:
        if wo.word == word1:
            check = False
            print("word is checked!")
            break
    term = 0;    
    if check:
        docposes=[]
        for doc in docs:
            countInDoc = 0
            checkInDoc = False
            postions = []
            temp = str(doc.main)
            postions = [i+1 for i,w in enumerate(temp.split()) if w == word1]
            if postions != "[]":
                if len(postions) != 0:
                    checkInDoc = True
                    countInDoc = len(postions)
                    term = term + countInDoc
                    if checkInDoc :
                        id = doc.id
                        docpos = [id,countInDoc,postions]
                        docposes.append(docpos)
        posInd = PosInd(word1,term,docposes)
        PositionIndexArray.append(posInd)

In [27]:
#create a positinal index class for each word
class PosInd:
    def __init__(self,word,abundance,docPositions):
        self.word = word
        self.abundance = abundance
        self.docPositions = docPositions

In [ ]:
#pass all the words for create inverted index
for i in allwords:
    i = remove_punctuation(i)
    addWordToWords(i)

In [25]:
#because of saving doc name in positinal index text and use that for finding information this method extract doc number uses doc name
def finddocnum(id):
    for doc in docs:
        if doc.id == id:
            return doc.num

In [28]:
#create posInd class and PositionIndexArray from saved posting lis
#create this list took 31 hours to create after one time cration i saved it and use that for saving time
with open("posting_list.txt" , 'r') as f:
    document = f.read()
word = ""
termfreq = ""
postintion = []
for line in document.split("\n"):
    if "word : " in line:
        word = line[7:]
    elif "docs : " in line:
        termfreq = line[39:]
    elif "positions : " in line:
        postintion = line[90:]
    if word != "" and postintion != [] and termfreq != "":
        posInd = PosInd(word,termfreq,postintion)
        PositionIndexArray.append(posInd)
        word = ""
        termfreq = ""
        postintion = []

In [339]:
#create text with tokenid , docid and term frequency in that doc
termfreqindoc = []
tokenid = 0
file = open("termfreq.txt" ,'w')
for i in PositionIndexArray:
    temp = str(i.docPositions)
    tokenid += 1
    for word in temp.split(", ["):
        if "'" in word:
            termfreq = []
            for w in word.split(","):
                w = w.replace("[","")
                w = w.replace("'","")
                termfreq.append(w)
            docnumber = finddocnum(termfreq[0])
            file.write(str(tokenid))
            file.write(" , ")
            file.write(str(docnumber))
            file.write(" , ")
            file.write(str(termfreq[1]))
            file.write("\n")
    file.write(".......................................................")
    file.write("\n")
            

In [30]:
#extract document frequency and document name for find docid
#extract 20 word with most doc frequncy and thieri id and number of doc
docfrequ = []
for i in InvertedIndexArray:
    docfrequ.append(i.abundance)
temp = []
temp = docfrequ.copy()
temp.sort()
new_temp = temp[-21:-1]
wordid = []
last20=0
docfrequ_wordid = []
for j in range(0,len(new_temp)):
    for i in range(0,len(docfrequ)):
        if new_temp[j] == docfrequ[i]:
            if last20 <20:
                m = [i,new_temp[j]]
                wordid.append(i)
                docfrequ_wordid.append(m)
                last20 += 1
checkid = 0
for i in InvertedIndexArray:
    if checkid in wordid:
        docfrequ_wordid.append(i.word) 
    checkid+=1
print(docfrequ_wordid)

[[2, 598], [6, 598], [44, 598], [56, 598], [68, 598], [141, 598], [175, 598], [280, 598], [295, 598], [394, 598], [410, 598], [476, 598], [488, 598], [493, 598], [511, 598], [719, 598], [724, 598], [738, 598], [776, 598], [784, 598], 'comfort', 'drive', 'ac', 'car', 'great', 'on', 'interior', 'love', 'int', 'ad', 'ex', 'ca', 'ov', 'up', 'it', 've', 'll', 'and', 'to', 'no']


In [ ]:
#create text file with tokenid and tokenstring and document frequency
with open("docfrequ.txt" ,'w') as f:
    tokenid = 0
    for i in InvertedIndexArray:
        f.write("tokenID : ")
        f.write(str(tokenid+1))
        f.write(" -----> tokenString : ")
        f.write(i.word)
        f.write(" -----> ‫‪DocumentFrequency‬‬ : ")
        f.write(str(i.ababundance))
        f.write("\n")
        tokenid+=1

In [31]:
#function for finding 20 of largest part in list
def Nnumberele(list1, N): 
    new_list = [] 
    for i in range(0, N):  
        max1 = 0
        for j in range(len(list1)):      
            if list1[j] > max1: 
                max1 = list1[j]; 
        list1.remove(max1); 
        new_list.append(max1) 
    return new_list

In [ ]:
#extract 20 word with most term frequency and number of repeat
x = 0
termfr20 = []
w = []
for i in PositionIndexArray:
    termfr20.append(i.abundance)
for i in range(len(termfr20)):
    termfr20[i] = int(termfr20[i])
temp = Nnumberele(termfr20,20)
for m in range(0,20):
    for i in PositionIndexArray:
        if int(i.abundance) == temp[m]:
            twen = [ i.word ,temp[m]]
            w.append(twen)  
print(w)

In [54]:
#extract all brands of car from our files
def car_brands():
    brands = []
    for path in paths:
        if path[61:] not in brands:
            brands.append(path[61:])
    print(brands)

In [64]:
#number of cars per years  and all cars
def numberOfCarsPerYear():
    i2007 =0
    i2008=0
    i2009=0
    for path in paths:
        docid = str(path[56:])
        if "2007" in docid:
            i2007 = i2007+1
        elif docid.__contains__('2008'):
            i2008 = i2008 + 1
        elif docid.__contains__('2009'):
            i2009 = i2009 + 1
    iallcar = i2007 + i2008 + i2009   
    stat = 'number of cars in 2007: '+str(i2007)+"\nnumber of cars in 2008: "+str(i2008)+"\nnumber of cars in 2009: "+str(i2009)+"\nnumber of all cars : "+str(iallcar)
    return stat

In [105]:
#number of comments per year and all year
def comment_per_year():
    c2007 = 0
    c2008 = 0
    c2009 = 0
    for doc in docs:
        docid = str(doc.id)
        docmain = str(doc.main)
        if "2007" in docid:   
            c2007 = len(re.findall('text', docmain))/2 + c2007
        if "2008" in docid:
            c2008 = len(re.findall('text', docmain))/2 + c2008
        if "2009" in docid:
            c2009 = len(re.findall('text', docmain))/2 + c2009
    c2007 = int(c2007)
    c2008 = int(c2008)
    c2009 = int(c2009)
    c_all = c2007 + c2008 + c2009
    stat = 'number of comment in 2007: '+str(c2007)+"\nnumber of comment in 2008: "+str(c2008)+"\nnumber of comment in 2009: "+str(c2009)+"\nnumber of comment in all years: "+str(c_all)
    return stat

In [108]:
#number od comments per year
def comment_per_car():
    nc_car = []
    for doc in docs:
        c2007 = 0
        docid = str(doc.id)
        docmain = str(doc.main)
        for word in brands:
            word = str(word)
            word = word.replace('[','')
            word = word.replace(']','')
            word = word.replace("'","")
            word = word.replace(",","")
            if word == docid[5:]:
                if "2007" in docid:
                    c2007 = len(re.findall('text', docmain))/2
                    c2007 = int(c2007)
                    comcar = [word,"2007",c2007]
                    nc_car.append(comcar)
                elif "2008" in docid:
                    c2007 = len(re.findall('text', docmain))/2
                    c2007 = int(c2007)
                    comcar = [word,"2008",c2007]
                    nc_car.append(comcar)
                elif "2009" in docid:
                    c2007 = len(re.findall('text', docmain))/2
                    c2007 = int(c2007)
                    comcar = [word,"2009",c2007]
                    nc_car.append(comcar)
    print(nc_car)

In [81]:
#number of comments for each car in all years
def comment_allcar():
    all_cars = []
    for word in brands:
        word = str(word)
        c2007 = 0
        word = word.replace('[','')
        word = word.replace(']','')
        word = word.replace("'","")
        word = word.replace(",","")
        for doc in docs:
            docid = str(doc.id)
            docmain = str(doc.main)
            if word == docid[5:]:
                c2007 = len(re.findall('text', docmain))/2 +c2007
                c2007 = int(c2007)
        nomcar = [word,c2007]
        all_cars.append(nomcar)
    print(all_cars)

In [ ]:
car_brands()

In [ ]:
numberOfCarsPerYear()

In [ ]:
comment_per_year()

In [ ]:
comment_per_car()

In [ ]:
comment_allcar()